# Sirene API Explorer

This notebook provides an interactive way to explore the French Sirene API using the `sirene-api-client` library.

## What is Sirene?

Sirene (Système d'Identification du Répertoire des Entreprises) is the French business registry that contains information about:
- **Legal entities** (Unité Légale) - companies, associations, etc.
- **Establishments** (Établissement) - physical locations of businesses
- **Succession links** - relationships between entities

## Getting Started

1. **Get your API token** from [api.insee.fr](https://api.insee.fr)
2. **Set your token** in the configuration cell below
3. **Run the cells** to explore the API interactively


## 1. Setup and Configuration


In [2]:
# Import required libraries
from typing import Any, Dict

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sirene_api_client.api.etablissement.find_by_siret import sync as find_by_siret_sync
from sirene_api_client.api.unite_legale.find_by_siren import (
    asyncio as find_by_siren_async,
)
from sirene_api_client.api.unite_legale.find_by_siren import sync as find_by_siren_sync

# Import the Sirene API client
from sirene_api_client.client import AuthenticatedClient

# Set up plotting style
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [3]:
# Configuration - Fixed authentication method
# Based on official OpenAPI spec: https://api-apimanager.insee.fr/portal/environments/DEFAULT/apis/2ba0e549-5587-3ef1-9082-99cd865de66f/pages/6548510e-c3e1-3099-be96-6edf02870699/content
API_BASE_URL = "https://api.insee.fr/api-sirene/3.11"

# 🔑 IMPORTANT: Replace with your actual API token from data.gouv.fr
API_TOKEN = "33cf9797-9541-4bd4-8aee-5354bec491ca"

# You can also set it as an environment variable:
# API_TOKEN = os.getenv("SIRENE_API_TOKEN", "YOUR_API_TOKEN_HERE")

if API_TOKEN == "YOUR_API_TOKEN_HERE":
    print("⚠️  WARNING: Please set your API token in the API_TOKEN variable above")
    print(
        "   Get your token from: https://www.data.gouv.fr/dataservices/api-sirene-open-data/"
    )
else:
    print("✅ API token configured")

# Create the authenticated client with correct API key authentication
# The INSEE API uses X-INSEE-Api-Key-Integration header, not Bearer token
client = AuthenticatedClient(
    base_url=API_BASE_URL,
    token=API_TOKEN,
    prefix="",  # No prefix for API key
    auth_header_name="X-INSEE-Api-Key-Integration",  # Correct header name
)

print(f"🔗 Connected to: {API_BASE_URL}")
print(
    "📚 Official documentation: https://www.data.gouv.fr/dataservices/api-sirene-open-data/"
)
print("🔑 Authentication: API Key in X-INSEE-Api-Key-Integration header")

✅ API token configured
🔗 Connected to: https://api.insee.fr/api-sirene/3.11
📚 Official documentation: https://www.data.gouv.fr/dataservices/api-sirene-open-data/
🔑 Authentication: API Key in X-INSEE-Api-Key-Integration header


In [4]:
response = find_by_siren_sync(
    client=client, siren="061500542", masquer_valeurs_nulles=True
)
response

ReponseUniteLegale(header=Header(statut=200, message='OK', total=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, debut=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, nombre=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur_suivant=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, additional_properties={}), unite_legale=UniteLegale(score=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, siren='061500542', statut_diffusion_unite_legale='O', unite_purgee_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, date_creation_unite_legale=datetime.date(1961, 1, 1), date_naissance_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, code_commune_naissance_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, code_pays_naissance_unite_legale=<sirene_api_cli

In [5]:
result = await find_by_siren_async(
    siren="552043002",
    client=client,
    masquer_valeurs_nulles=True,  # This is the key difference!
)

In [6]:
result

ReponseUniteLegale(header=Header(statut=200, message='OK', total=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, debut=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, nombre=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur_suivant=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, additional_properties={}), unite_legale=UniteLegale(score=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, siren='552043002', statut_diffusion_unite_legale='O', unite_purgee_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, date_creation_unite_legale=datetime.date(1955, 1, 1), date_naissance_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, code_commune_naissance_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, code_pays_naissance_unite_legale=<sirene_api_cli

In [7]:
# Automatic Information Discovery - Recursive Data Explorer


def discover_nested_info(
    obj: Any, path: str = "", max_depth: int = 5, current_depth: int = 0
) -> Dict[str, Any]:
    """
    Recursively discover all information and nested structures in an object

    Args:
        obj: Object to explore
        path: Current path in the object hierarchy
        max_depth: Maximum depth to explore
        current_depth: Current exploration depth

    Returns:
        Dictionary with discovered information
    """
    if current_depth >= max_depth:
        return {"truncated": f"Max depth {max_depth} reached"}

    discovered = {}

    if hasattr(obj, "__dict__"):
        # Object with attributes
        for attr_name in dir(obj):
            if not attr_name.startswith("_") and not callable(
                getattr(obj, attr_name, None)
            ):
                try:
                    attr_value = getattr(obj, attr_name)
                    if attr_value is not None:
                        current_path = f"{path}.{attr_name}" if path else attr_name

                        if isinstance(attr_value, (str, int, float, bool, type(None))):
                            discovered[current_path] = attr_value
                        elif isinstance(attr_value, (list, tuple)):
                            discovered[current_path] = (
                                f"List/Tuple with {len(attr_value)} items"
                            )
                            if attr_value and len(attr_value) <= 3:  # Show small lists
                                for i, item in enumerate(attr_value):
                                    item_path = f"{current_path}[{i}]"
                                    if isinstance(
                                        item, (str, int, float, bool, type(None))
                                    ):
                                        discovered[item_path] = item
                                    else:
                                        nested = discover_nested_info(
                                            item,
                                            item_path,
                                            max_depth,
                                            current_depth + 1,
                                        )
                                        discovered.update(nested)
                        elif hasattr(attr_value, "__dict__"):
                            # Nested object
                            nested = discover_nested_info(
                                attr_value, current_path, max_depth, current_depth + 1
                            )
                            discovered.update(nested)
                        else:
                            discovered[current_path] = str(type(attr_value))

                except Exception as e:
                    discovered[f"{path}.{attr_name}" if path else attr_name] = (
                        f"Error accessing: {e}"
                    )

    elif isinstance(obj, dict):
        # Dictionary
        for key, value in obj.items():
            current_path = f"{path}.{key}" if path else key

            if isinstance(value, (str, int, float, bool, type(None))):
                discovered[current_path] = value
            elif isinstance(value, (list, tuple)):
                discovered[current_path] = f"List/Tuple with {len(value)} items"
                if value and len(value) <= 3:  # Show small lists
                    for i, item in enumerate(value):
                        item_path = f"{current_path}[{i}]"
                        if isinstance(item, (str, int, float, bool, type(None))):
                            discovered[item_path] = item
                        else:
                            nested = discover_nested_info(
                                item, item_path, max_depth, current_depth + 1
                            )
                            discovered.update(nested)
            elif isinstance(value, dict):
                nested = discover_nested_info(
                    value, current_path, max_depth, current_depth + 1
                )
                discovered.update(nested)
            else:
                discovered[current_path] = str(type(value))

    elif isinstance(obj, (list, tuple)):
        # List or tuple
        discovered[f"{path}_count"] = len(obj)
        for i, item in enumerate(obj):
            item_path = f"{path}[{i}]"
            if isinstance(item, (str, int, float, bool, type(None))):
                discovered[item_path] = item
            else:
                nested = discover_nested_info(
                    item, item_path, max_depth, current_depth + 1
                )
                discovered.update(nested)

    else:
        # Primitive type
        discovered[path] = obj

    return discovered


def explore_siren_auto_discovery(siren: str, show_raw: bool = False):
    """
    Automatically discover all information available for a SIREN

    Args:
        siren: 9-digit SIREN code
        show_raw: Whether to show raw data structure
    """
    print(f"🔍 Automatic SIREN Discovery: {siren}")
    print("=" * 80)

    try:
        response = find_by_siren_sync(
            client=client, siren=siren, masquer_valeurs_nulles=True
        )

        if not response:
            print("❌ No response received")
            return None

        print(f"📋 Response Type: {type(response).__name__}")
        print(f"📋 Response Module: {type(response).__module__}")

        # Discover response structure
        print("\n🔍 RESPONSE STRUCTURE DISCOVERY")
        print("-" * 50)
        response_info = discover_nested_info(response, "response", max_depth=3)

        for path, value in sorted(response_info.items()):
            if isinstance(value, str) and len(value) > 100:
                print(f"   {path}: {value[:100]}...")
            else:
                print(f"   {path}: {value}")

        if hasattr(response, "unite_legale") and response.unite_legale:
            unite_legale = response.unite_legale

            print("\n🏢 UNITE LEGALE STRUCTURE DISCOVERY")
            print("-" * 50)
            print(f"   Type: {type(unite_legale).__name__}")
            print(f"   Module: {type(unite_legale).__module__}")

            # Discover unite_legale structure
            unite_info = discover_nested_info(unite_legale, "unite_legale", max_depth=4)

            # Group information by type
            basic_fields = {}
            period_fields = {}
            personal_fields = {}
            business_fields = {}
            other_fields = {}

            for path, value in unite_info.items():
                path_lower = path.lower()
                if any(
                    keyword in path_lower
                    for keyword in ["siren", "date", "creation", "nombre", "statut"]
                ):
                    basic_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in [
                        "periode",
                        "denomination",
                        "categorie",
                        "activite",
                        "etat",
                    ]
                ):
                    period_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in ["prenom", "nom", "naissance", "sexe", "nationalite"]
                ):
                    personal_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in ["effectif", "entreprise", "association", "sigle"]
                ):
                    business_fields[path] = value
                else:
                    other_fields[path] = value

            # Display grouped information
            if basic_fields:
                print(f"\n📋 BASIC INFORMATION ({len(basic_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(basic_fields.items()):
                    print(f"   {path}: {value}")

            if period_fields:
                print(f"\n🏢 COMPANY/PERIOD INFORMATION ({len(period_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(period_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            if personal_fields:
                print(f"\n👤 PERSONAL INFORMATION ({len(personal_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(personal_fields.items()):
                    print(f"   {path}: {value}")

            if business_fields:
                print(f"\n📊 BUSINESS METRICS ({len(business_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(business_fields.items()):
                    print(f"   {path}: {value}")

            if other_fields:
                print(f"\n🔍 OTHER FIELDS ({len(other_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(other_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            # Summary statistics
            print("\n📈 DISCOVERY SUMMARY")
            print("-" * 30)
            print(f"   Total fields discovered: {len(unite_info)}")
            print(f"   Basic fields: {len(basic_fields)}")
            print(f"   Company/Period fields: {len(period_fields)}")
            print(f"   Personal fields: {len(personal_fields)}")
            print(f"   Business fields: {len(business_fields)}")
            print(f"   Other fields: {len(other_fields)}")

            # Show raw structure if requested
            if show_raw:
                print("\n🔍 RAW DATA STRUCTURE")
                print("-" * 30)
                try:
                    raw_data = unite_legale.to_dict()
                    print(f"   Raw data type: {type(raw_data)}")
                    print(f"   Raw data keys: {list(raw_data.keys())}")

                    # Show sample of raw data
                    for key, value in list(raw_data.items())[:5]:
                        print(
                            f"   {key}: {str(value)[:100]}{'...' if len(str(value)) > 100 else ''}"
                        )

                    if len(raw_data) > 5:
                        print(f"   ... and {len(raw_data) - 5} more fields")

                except Exception as e:
                    print(f"   Error accessing raw data: {e}")

            return {
                "response_info": response_info,
                "unite_info": unite_info,
                "basic_fields": basic_fields,
                "period_fields": period_fields,
                "personal_fields": personal_fields,
                "business_fields": business_fields,
                "other_fields": other_fields,
                "summary": {
                    "total_fields": len(unite_info),
                    "basic_count": len(basic_fields),
                    "period_count": len(period_fields),
                    "personal_count": len(personal_fields),
                    "business_count": len(business_fields),
                    "other_count": len(other_fields),
                },
            }
        else:
            print("❌ No unite_legale found in response")
            return None

    except Exception as e:
        print(f"❌ Error during discovery: {e}")
        import traceback

        traceback.print_exc()
        return None


# Example usage
print("🚀 Automatic SIREN Information Discovery")
print("=" * 80)

# Test with Air France SIREN
air_france_discovery = explore_siren_auto_discovery("552049447", show_raw=True)

print("\n" + "=" * 80)
print("💡 Usage Examples:")
print("   explore_siren_auto_discovery('123456789')  # Basic discovery")
print("   explore_siren_auto_discovery('123456789', show_raw=True)  # With raw data")

🚀 Automatic SIREN Information Discovery
🔍 Automatic SIREN Discovery: 552049447
📋 Response Type: ReponseUniteLegale
📋 Response Module: sirene_api_client.models.reponse_unite_legale

🔍 RESPONSE STRUCTURE DISCOVERY
--------------------------------------------------
   response: ReponseUniteLegale(header=Header(statut=200, message='OK', total=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, debut=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, nombre=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur_suivant=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, additional_properties={}), unite_legale=UniteLegale(score=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, siren='552049447', statut_diffusion_unite_legale='O', unite_purgee_unite_legale=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, date_creation_unite_le

## 2. API Service Information

Let's start by checking the API service status and getting information about the service.


In [8]:
# Automatic SIRET Information Discovery - Recursive Data Explorer
def explore_siret_auto_discovery(siret: str, show_raw: bool = False):
    """
    Automatically discover all information available for a SIRET (establishment)

    Args:
        siret: 14-digit SIRET code
        show_raw: Whether to show raw data structure
    """
    print(f"🔍 Automatic SIRET Discovery: {siret}")
    print("=" * 80)

    try:
        response = find_by_siret_sync(
            client=client, siret=siret, masquer_valeurs_nulles=True
        )

        if not response:
            print("❌ No response received")
            return None

        print(f"📋 Response Type: {type(response).__name__}")
        print(f"📋 Response Module: {type(response).__module__}")

        # Discover response structure
        print("\n🔍 RESPONSE STRUCTURE DISCOVERY")
        print("-" * 50)
        response_info = discover_nested_info(response, "response", max_depth=3)

        for path, value in sorted(response_info.items()):
            if isinstance(value, str) and len(value) > 100:
                print(f"   {path}: {value[:100]}...")
            else:
                print(f"   {path}: {value}")

        if hasattr(response, "etablissement") and response.etablissement:
            etablissement = response.etablissement

            print("\n🏢 ETABLISSEMENT STRUCTURE DISCOVERY")
            print("-" * 50)
            print(f"   Type: {type(etablissement).__name__}")
            print(f"   Module: {type(etablissement).__module__}")

            # Discover etablissement structure
            etablissement_info = discover_nested_info(
                etablissement, "etablissement", max_depth=4
            )

            # Group information by type for establishments
            basic_fields = {}
            address_fields = {}
            activity_fields = {}
            legal_fields = {}
            business_fields = {}
            other_fields = {}

            for path, value in etablissement_info.items():
                path_lower = path.lower()
                if any(
                    keyword in path_lower
                    for keyword in [
                        "siret",
                        "siren",
                        "nic",
                        "date",
                        "creation",
                        "statut",
                    ]
                ):
                    basic_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in [
                        "adresse",
                        "adresse",
                        "commune",
                        "code_postal",
                        "pays",
                        "region",
                    ]
                ):
                    address_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in ["activite", "nomenclature", "ape", "naf"]
                ):
                    activity_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in ["denomination", "enseigne", "nom", "sigle"]
                ):
                    legal_fields[path] = value
                elif any(
                    keyword in path_lower
                    for keyword in ["effectif", "caractere", "employeur", "etat"]
                ):
                    business_fields[path] = value
                else:
                    other_fields[path] = value

            # Display grouped information
            if basic_fields:
                print(f"\n📋 BASIC INFORMATION ({len(basic_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(basic_fields.items()):
                    print(f"   {path}: {value}")

            if address_fields:
                print(f"\n📍 ADDRESS INFORMATION ({len(address_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(address_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            if activity_fields:
                print(f"\n🏭 ACTIVITY INFORMATION ({len(activity_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(activity_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            if legal_fields:
                print(f"\n⚖️ LEGAL/NAMING INFORMATION ({len(legal_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(legal_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            if business_fields:
                print(f"\n📊 BUSINESS METRICS ({len(business_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(business_fields.items()):
                    print(f"   {path}: {value}")

            if other_fields:
                print(f"\n🔍 OTHER FIELDS ({len(other_fields)} fields)")
                print("-" * 30)
                for path, value in sorted(other_fields.items()):
                    if isinstance(value, str) and len(value) > 80:
                        print(f"   {path}: {value[:80]}...")
                    else:
                        print(f"   {path}: {value}")

            # Summary statistics
            print("\n📈 DISCOVERY SUMMARY")
            print("-" * 30)
            print(f"   Total fields discovered: {len(etablissement_info)}")
            print(f"   Basic fields: {len(basic_fields)}")
            print(f"   Address fields: {len(address_fields)}")
            print(f"   Activity fields: {len(activity_fields)}")
            print(f"   Legal/Naming fields: {len(legal_fields)}")
            print(f"   Business fields: {len(business_fields)}")
            print(f"   Other fields: {len(other_fields)}")

            # Show raw structure if requested
            if show_raw:
                print("\n🔍 RAW DATA STRUCTURE")
                print("-" * 30)
                try:
                    raw_data = etablissement.to_dict()
                    print(f"   Raw data type: {type(raw_data)}")
                    print(f"   Raw data keys: {list(raw_data.keys())}")

                    # Show sample of raw data
                    for key, value in list(raw_data.items())[:5]:
                        print(
                            f"   {key}: {str(value)[:100]}{'...' if len(str(value)) > 100 else ''}"
                        )

                    if len(raw_data) > 5:
                        print(f"   ... and {len(raw_data) - 5} more fields")

                except Exception as e:
                    print(f"   Error accessing raw data: {e}")

            return {
                "response_info": response_info,
                "etablissement_info": etablissement_info,
                "basic_fields": basic_fields,
                "address_fields": address_fields,
                "activity_fields": activity_fields,
                "legal_fields": legal_fields,
                "business_fields": business_fields,
                "other_fields": other_fields,
                "summary": {
                    "total_fields": len(etablissement_info),
                    "basic_count": len(basic_fields),
                    "address_count": len(address_fields),
                    "activity_count": len(activity_fields),
                    "legal_count": len(legal_fields),
                    "business_count": len(business_fields),
                    "other_count": len(other_fields),
                },
            }
        else:
            print("❌ No etablissement found in response")
            return None

    except Exception as e:
        print(f"❌ Error during discovery: {e}")
        import traceback

        traceback.print_exc()
        return None


# Example usage
print("🚀 Automatic SIRET Information Discovery")
print("=" * 80)

# Test with Air France SIRET (example - you may need to find a valid one)
# Note: SIRET codes are 14 digits, combining SIREN (9) + NIC (5)
try:
    air_france_siret_discovery = explore_siret_auto_discovery(
        "55204944700019", show_raw=True
    )
except Exception as e:
    print(f"❌ Error with Air France SIRET: {e}")
    print("💡 Trying with a different SIRET...")

    # Try with a different SIRET if the first one fails
    try:
        # You can replace this with any valid 14-digit SIRET
        test_siret_discovery = explore_siret_auto_discovery(
            "12345678901234", show_raw=True
        )
    except Exception as e2:
        print(f"❌ Error with test SIRET: {e2}")
        print("💡 Please provide a valid 14-digit SIRET code")

print("\n" + "=" * 80)
print("💡 Usage Examples:")
print("   explore_siret_auto_discovery('12345678901234')  # Basic discovery")
print(
    "   explore_siret_auto_discovery('12345678901234', show_raw=True)  # With raw data"
)
print("\n📝 Note: SIRET = SIREN (9 digits) + NIC (5 digits) = 14 digits total")

🚀 Automatic SIRET Information Discovery
🔍 Automatic SIRET Discovery: 55204944700019
📋 Response Type: ReponseErreur
📋 Response Module: sirene_api_client.models.reponse_erreur

🔍 RESPONSE STRUCTURE DISCOVERY
--------------------------------------------------
   response: ReponseErreur(header=Header(statut=404, message='Aucun élément trouvé pour le siret 55204944700019', total=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, debut=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, nombre=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, curseur_suivant=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, additional_properties={}), additional_properties={})
❌ No etablissement found in response

💡 Usage Examples:
   explore_siret_auto_discovery('12345678901234')  # Basic discovery
   explore_siret_auto_discovery('12345678901234', show_raw=True)  # With raw d

In [9]:
# Test: Check what fields are returned by find_by_siren_sync
siren_to_search = "061500542"  # Air France example

print(f"🔍 Testing SIREN: {siren_to_search}")

try:
    response = find_by_siren_sync(
        client=client,
        siren=siren_to_search,
        masquer_valeurs_nulles=True,  # Hide null values
    )

    print(f"\n📋 Response Type: {type(response)}")
    print(f"📋 Response Attributes: {dir(response)}")

    if response and hasattr(response, "unite_legale"):
        unite_legale = response.unite_legale

        print(f"\n🏢 UniteLegale Type: {type(unite_legale)}")
        print(f"🏢 UniteLegale Attributes: {dir(unite_legale)}")

        print("\n🏢 Legal Entity Information:")
        print(f"   SIREN: {unite_legale.siren}")
        print(f"   Creation Date: {unite_legale.date_creation_unite_legale}")
        print(f"   Number of Periods: {unite_legale.nombre_periodes_unite_legale}")

        # Check if periods exist and get current company information
        if (
            unite_legale.periodes_unite_legale
            and len(unite_legale.periodes_unite_legale) > 0
        ):
            print(
                f"   Number of Periods Available: {len(unite_legale.periodes_unite_legale)}"
            )

            # Get the most recent period (last in the list)
            current_period = unite_legale.periodes_unite_legale[-1]
            print(f"   Current Period Type: {type(current_period)}")
            print(f"   Current Period Attributes: {dir(current_period)}")

            print(f"   Company Name: {current_period.denomination_unite_legale}")
            print(
                f"   Legal Category: {current_period.categorie_juridique_unite_legale}"
            )
            print(f"   Activity: {current_period.activite_principale_unite_legale}")
            print(
                f"   Administrative Status: {current_period.etat_administratif_unite_legale}"
            )
            print(f"   Period Start: {current_period.date_debut}")
            print(f"   Period End: {current_period.date_fin}")

            # Display alternative names if available
            if current_period.denomination_usuelle_1_unite_legale:
                print(
                    f"   Usual Name 1: {current_period.denomination_usuelle_1_unite_legale}"
                )
            if current_period.denomination_usuelle_2_unite_legale:
                print(
                    f"   Usual Name 2: {current_period.denomination_usuelle_2_unite_legale}"
                )
            if current_period.denomination_usuelle_3_unite_legale:
                print(
                    f"   Usual Name 3: {current_period.denomination_usuelle_3_unite_legale}"
                )
        else:
            print("   No period information available")

        # Display full legal entity data
        print("\n📋 Full Legal Entity Data:")
        legal_entity_data = unite_legale.to_dict()

        # Create a DataFrame for better visualization
        df_data = []
        for key, value in legal_entity_data.items():
            if value is not None and value != "":
                df_data.append({"Field": key, "Value": str(value)})

        df = pd.DataFrame(df_data)
        print(df.to_string(index=False))

    else:
        print("❌ No legal entity found or error occurred")
        if response:
            print(f"Response: {response}")

except Exception as e:
    print(f"❌ Error: {e}")
    # Try to get more detailed error information
    if "not a valid" in str(e):
        print(
            "💡 This error suggests the API returned data that doesn't match our model."
        )
        print(
            "   This can happen when the API schema changes or returns unexpected values."
        )
        print("   Try using a different SIREN or check the API documentation.")

🔍 Testing SIREN: 061500542

📋 Response Type: <class 'sirene_api_client.models.reponse_unite_legale.ReponseUniteLegale'>
📋 Response Attributes: ['__annotations__', '__attrs_attrs__', '__attrs_own_setattr__', '__attrs_props__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__replace__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', 'additional_keys', 'additional_properties', 'from_dict', 'header', 'to_dict', 'unite_legale']

🏢 UniteLegale Type: <class 'sirene_api_client.models.unite_legale.UniteLegale'>
🏢 UniteLegale Attributes: ['__annotations__', '__attrs_attrs__', '__attrs_own_setatt

In [10]:
# 🎯 COMPREHENSIVE SIREN INFORMATION RETRIEVAL
# =================================================================================
# Get ALL available information for SIREN 061500542

# Import necessary classes
from sirene_api_client.models.reponse_unite_legale import ReponseUniteLegale
from sirene_api_client.api_types import UNSET


def get_comprehensive_siren_info(siren: str, show_raw: bool = False):
    """
    Get comprehensive information for a SIREN including:
    - Legal unit information (unité légale)
    - All establishments (établissements)
    - Historical periods
    - Complete data structure

    Args:
        siren: 9-digit SIREN code
        show_raw: Whether to show raw JSON data
    """
    print(f"🔍 COMPREHENSIVE SIREN ANALYSIS: {siren}")
    print("=" * 80)

    try:
        # 1. Get Legal Unit Information (Unité Légale)
        print("📋 STEP 1: Legal Unit Information (Unité Légale)")
        print("-" * 50)

        unite_legale_response = find_by_siren_sync(
            siren=siren,
            client=client,
            masquer_valeurs_nulles=False,  # Show all fields, even null ones
        )

        if isinstance(unite_legale_response, ReponseUniteLegale):
            print(f"✅ Legal unit found: {unite_legale_response.unite_legale.siren}")
            print(
                f"📊 Status: {unite_legale_response.unite_legale.statut_diffusion_unite_legale}"
            )
            print(
                f"📅 Creation date: {unite_legale_response.unite_legale.date_creation_unite_legale}"
            )

            # Display key information
            ul = unite_legale_response.unite_legale
            print(f"\n🏢 COMPANY DETAILS:")
            print(f"   • SIREN: {ul.siren}")
            print(f"   • Legal name: {getattr(ul, 'denomination_unite_legale', 'N/A')}")
            print(
                f"   • Legal category: {getattr(ul, 'categorie_juridique_unite_legale', 'N/A')}"
            )
            print(
                f"   • Activity: {getattr(ul, 'activite_principale_unite_legale', 'N/A')}"
            )
            print(
                f"   • Employee count: {getattr(ul, 'tranche_effectifs_unite_legale', 'N/A')}"
            )
            print(
                f"   • Last update: {getattr(ul, 'date_dernier_traitement_unite_legale', 'N/A')}"
            )

            # Show all available fields
            print(f"\n📋 ALL AVAILABLE FIELDS:")
            print("-" * 30)
            for field_name in dir(ul):
                if not field_name.startswith("_") and not callable(
                    getattr(ul, field_name)
                ):
                    value = getattr(ul, field_name)
                    if value is not None and value != UNSET:
                        print(f"   • {field_name}: {value}")

            # 2. Get Establishments Information
            print(f"\n🏭 STEP 2: Establishments Information (Établissements)")
            print("-" * 50)

            # Import establishment search functions
            from sirene_api_client.api.etablissement.find_by_siret import (
                sync as find_by_siret_sync,
            )

            # Search for establishments using the SIREN as base
            # Note: We'll search for establishments that start with this SIREN
            print(f"🔍 Searching for establishments with SIREN base: {siren}")

            # Try to find establishments by searching with the SIREN
            # This is a bit tricky as we need to know the NIC codes
            # Let's try a few common patterns
            common_nics = ["00001", "00002", "00003", "00004", "00005"]
            establishments_found = []

            for nic in common_nics:
                siret = siren + nic
                try:
                    etablissement_response = find_by_siret_sync(
                        siret=siret, client=client, masquer_valeurs_nulles=False
                    )
                    if (
                        hasattr(etablissement_response, "etablissement")
                        and etablissement_response.etablissement
                    ):
                        establishments_found.append(etablissement_response)
                        print(f"✅ Found establishment: {siret}")
                except Exception as e:
                    # SIRET not found, continue
                    pass

            if establishments_found:
                print(f"\n📊 ESTABLISHMENTS SUMMARY:")
                print(f"   • Total found: {len(establishments_found)}")
                for i, etab in enumerate(establishments_found, 1):
                    e = etab.etablissement
                    print(f"   • Establishment {i}: {e.siret}")
                    print(
                        f"     - Address: {getattr(e, 'adresse_etablissement', 'N/A')}"
                    )
                    print(
                        f"     - Activity: {getattr(e, 'activite_principale_etablissement', 'N/A')}"
                    )
                    print(
                        f"     - Status: {getattr(e, 'etat_administratif_etablissement', 'N/A')}"
                    )
            else:
                print("❌ No establishments found with common NIC patterns")
                print("💡 Try searching manually with specific SIRET codes")

            # 3. Show Raw Data Structure (if requested)
            if show_raw:
                print(f"\n🔍 RAW DATA STRUCTURE")
                print("-" * 30)
                try:
                    raw_data = ul.to_dict()
                    print(f"Raw data type: {type(raw_data)}")
                    print(f"Raw data keys: {list(raw_data.keys())}")
                    print(f"\nRaw JSON (first 1000 chars):")
                    import json

                    json_str = json.dumps(raw_data, indent=2, default=str)
                    print(json_str[:1000] + "..." if len(json_str) > 1000 else json_str)
                except Exception as e:
                    print(f"❌ Error getting raw data: {e}")

            # 4. Summary
            print(f"\n📈 COMPREHENSIVE ANALYSIS SUMMARY")
            print("-" * 40)
            print(f"✅ Legal unit: Found and analyzed")
            print(f"✅ Establishments: {len(establishments_found)} found")
            print(f"✅ Data completeness: High")
            print(
                f"✅ Last update: {getattr(ul, 'date_dernier_traitement_unite_legale', 'N/A')}"
            )

            return {
                "legal_unit": unite_legale_response,
                "establishments": establishments_found,
                "siren": siren,
            }

        else:
            print(f"❌ Error retrieving legal unit: {unite_legale_response}")
            return None

    except Exception as e:
        print(f"❌ Error in comprehensive analysis: {e}")
        return None


# Test with your SIREN
print("🚀 COMPREHENSIVE SIREN INFORMATION RETRIEVAL")
print("=" * 80)

# Get all information for SIREN 061500542
siren_info = get_comprehensive_siren_info("061500542", show_raw=False)

if siren_info:
    print(f"\n🎉 SUCCESS! Retrieved comprehensive information for SIREN 061500542")
    print(f"📊 Legal unit data: Available")
    print(f"🏭 Establishments data: {len(siren_info['establishments'])} found")
else:
    print(f"\n❌ FAILED to retrieve information for SIREN 061500542")

🚀 COMPREHENSIVE SIREN INFORMATION RETRIEVAL
🔍 COMPREHENSIVE SIREN ANALYSIS: 061500542
📋 STEP 1: Legal Unit Information (Unité Légale)
--------------------------------------------------
✅ Legal unit found: 061500542
📊 Status: O
📅 Creation date: 1961-01-01

🏢 COMPANY DETAILS:
   • SIREN: 061500542
   • Legal name: N/A
   • Legal category: N/A
   • Activity: N/A
   • Employee count: 32
   • Last update: 2025-02-26T11:46:01.173

📋 ALL AVAILABLE FIELDS:
------------------------------
   • additional_keys: []
   • additional_properties: {}
   • annee_categorie_entreprise: 2023
   • annee_effectifs_unite_legale: 2023
   • categorie_entreprise: ETI
   • date_creation_unite_legale: 1961-01-01
   • date_dernier_traitement_unite_legale: 2025-02-26T11:46:01.173
   • nombre_periodes_unite_legale: 10
   • periodes_unite_legale: [PeriodeUniteLegale(date_fin=<sirene_api_client.api_types.Unset object at 0x000001F76E2BC6E0>, date_debut=datetime.date(2024, 9, 1), etat_administratif_unite_legale=<PeriodeU

In [11]:
# 🔍 GET ALL SIRET CODES FROM SIREN
# =================================================================================
# Better approach: Use establishment search to find all SIRET codes for a SIREN


def get_all_siret_from_siren(siren: str):
    """
    Get all SIRET codes associated with a SIREN by searching establishments

    Args:
        siren: 9-digit SIREN code

    Returns:
        List of SIRET codes and their details
    """
    print(f"🔍 FINDING ALL SIRET CODES FOR SIREN: {siren}")
    print("=" * 60)

    try:
        # Import the establishment search function
        from sirene_api_client.api.etablissement.find_by_post_etablissement import (
            sync as find_by_post_etablissement_sync,
        )

        # Search for establishments using the SIREN
        # We'll use a POST request with search criteria
        search_criteria = {
            "q": f"siren:{siren}",  # Search by SIREN
            "nombre": 100,  # Get up to 100 results
            "masquerValeursNulles": False,
        }

        print(f"📋 Searching establishments with criteria: {search_criteria}")

        # Make the search request
        response = find_by_post_etablissement_sync(client=client, **search_criteria)

        if hasattr(response, "etablissements") and response.etablissements:
            establishments = response.etablissements
            print(f"✅ Found {len(establishments)} establishments")

            siret_list = []
            print(f"\n📊 ALL SIRET CODES FOR SIREN {siren}:")
            print("-" * 50)

            for i, etab in enumerate(establishments, 1):
                siret = etab.siret
                nic = siret[-5:]  # Last 5 digits are the NIC

                print(f"   {i:2d}. SIRET: {siret} (NIC: {nic})")
                print(
                    f"       • Name: {getattr(etab, 'denomination_usuelle_etablissement', 'N/A')}"
                )
                print(
                    f"       • Address: {getattr(etab, 'adresse_etablissement', 'N/A')}"
                )
                print(
                    f"       • Activity: {getattr(etab, 'activite_principale_etablissement', 'N/A')}"
                )
                print(
                    f"       • Status: {getattr(etab, 'etat_administratif_etablissement', 'N/A')}"
                )
                print(
                    f"       • Creation: {getattr(etab, 'date_creation_etablissement', 'N/A')}"
                )
                print()

                siret_list.append({"siret": siret, "nic": nic, "etablissement": etab})

            print(f"📈 SUMMARY:")
            print(f"   • SIREN: {siren}")
            print(f"   • Total SIRET codes found: {len(siret_list)}")
            print(f"   • NIC codes: {[s['nic'] for s in siret_list]}")

            return siret_list

        else:
            print(f"❌ No establishments found for SIREN {siren}")
            print(f"Response: {response}")
            return []

    except Exception as e:
        print(f"❌ Error searching establishments: {e}")
        return []


# Test with SIREN 061500542
print("🚀 SIRET DISCOVERY FROM SIREN")
print("=" * 60)

siret_codes = get_all_siret_from_siren("061500542")

if siret_codes:
    print(f"\n🎉 SUCCESS! Found {len(siret_codes)} SIRET codes")
    print(f"📋 Complete list:")
    for siret_info in siret_codes:
        print(f"   • {siret_info['siret']} (NIC: {siret_info['nic']})")
else:
    print(f"\n❌ No SIRET codes found for SIREN 061500542")

🚀 SIRET DISCOVERY FROM SIREN
🔍 FINDING ALL SIRET CODES FOR SIREN: 061500542
📋 Searching establishments with criteria: {'q': 'siren:061500542', 'nombre': 100, 'masquerValeursNulles': False}
❌ Error searching establishments: sync() got an unexpected keyword argument 'q'

❌ No SIRET codes found for SIREN 061500542


In [12]:
# 🎯 USE EXISTING ETL SYSTEM TO GET ALL SIRET CODES FROM SIREN
# =================================================================================
# The project already has a complete ETL system for this!

from sirene_api_client.etl import extract_and_transform_siren
import asyncio


async def get_all_siret_from_siren_etl(siren: str):
    """
    Use the existing ETL system to get all SIRET codes from a SIREN

    Args:
        siren: 9-digit SIREN code

    Returns:
        Complete SIREN data including all establishments
    """
    print(f"🔍 USING EXISTING ETL SYSTEM FOR SIREN: {siren}")
    print("=" * 60)

    try:
        # Use the existing ETL function
        result = await extract_and_transform_siren(siren, client)

        print(f"✅ ETL extraction completed successfully!")
        print(f"📊 Company: {result.company.name}")
        print(f"📊 Total facilities found: {len(result.facilities)}")

        # Extract SIRET codes
        siret_codes = []
        print(f"\n📋 ALL SIRET CODES FOR SIREN {siren}:")
        print("-" * 50)

        for i, facility in enumerate(result.facilities, 1):
            # Get SIRET from identifiers
            siret_identifier = None
            for identifier in facility.identifiers:
                if identifier.scheme == "siret":
                    siret_identifier = identifier.value
                    break

            if siret_identifier:
                siret = siret_identifier
                nic = siret[-5:]  # Last 5 digits are the NIC

                print(f"   {i:2d}. SIRET: {siret} (NIC: {nic})")
                print(f"       • Name: {facility.name}")
                print(f"       • Parent SIREN: {facility.parent_siren}")
                print(f"       • NIC: {facility.nic}")
                print(f"       • Creation: {facility.creation_date}")
                print(f"       • Headquarters: {facility.is_headquarters}")
                print()

                siret_codes.append({"siret": siret, "nic": nic, "facility": facility})

        print(f"📈 SUMMARY:")
        print(f"   • SIREN: {siren}")
        print(f"   • Company: {result.company.name}")
        print(f"   • Total SIRET codes found: {len(siret_codes)}")
        print(f"   • NIC codes: {[s['nic'] for s in siret_codes]}")

        return {
            "company": result.company,
            "facilities": result.facilities,
            "siret_codes": siret_codes,
            "siren": siren,
        }

    except Exception as e:
        print(f"❌ Error in ETL extraction: {e}")
        return None


# Test with SIREN 061500542
print("🚀 USING EXISTING ETL SYSTEM")
print("=" * 60)

# For Jupyter notebooks, we need to await the async function directly
# instead of using asyncio.run()
siren_data = await get_all_siret_from_siren_etl("061500542")

if siren_data:
    print(
        f"\n🎉 SUCCESS! ETL system found {len(siren_data['siret_codes'])} SIRET codes"
    )
    print(f"📋 Complete list:")
    for siret_info in siren_data["siret_codes"]:
        print(f"   • {siret_info['siret']} (NIC: {siret_info['nic']})")
else:
    print(f"\n❌ ETL extraction failed for SIREN 061500542")

🚀 USING EXISTING ETL SYSTEM
🔍 USING EXISTING ETL SYSTEM FOR SIREN: 061500542
✅ ETL extraction completed successfully!
📊 Company: BDO RHONE-ALPES
📊 Total facilities found: 33

📋 ALL SIRET CODES FOR SIREN 061500542:
--------------------------------------------------
    1. SIRET: 06150054200040 (NIC: 00040)
       • Name: Unknown Facility
       • Parent SIREN: 061500542
       • NIC: 00040
       • Creation: 1900-01-01
       • Headquarters: False

    2. SIRET: 06150054200057 (NIC: 00057)
       • Name: BDO
       • Parent SIREN: 061500542
       • NIC: 00057
       • Creation: 1900-01-01
       • Headquarters: False

    3. SIRET: 06150054200263 (NIC: 00263)
       • Name: Unknown Facility
       • Parent SIREN: 061500542
       • NIC: 00263
       • Creation: 2009-10-01
       • Headquarters: False

    4. SIRET: 06150054200149 (NIC: 00149)
       • Name: Unknown Facility
       • Parent SIREN: 061500542
       • NIC: 00149
       • Creation: 1992-06-01
       • Headquarters: False



In [13]:
# 🔧 FIXED FACILITY NAME EXTRACTION
# =================================================================================
# The ETL system has a bug - it's not using the uniteLegale.denominationUniteLegale
# Let's create a corrected version that shows the proper facility names


def get_all_siret_from_siren_corrected(siren: str):
    """
    Get all SIRET codes with CORRECT facility names from the raw API response

    Args:
        siren: 9-digit SIREN code

    Returns:
        Complete SIREN data with corrected facility names
    """
    print(f"🔍 CORRECTED SIRET DISCOVERY FOR SIREN: {siren}")
    print("=" * 60)

    try:
        # Import the establishment search function
        from sirene_api_client.api.etablissement.find_by_post_etablissement import (
            sync as find_by_post_etablissement_sync,
        )
        from sirene_api_client.api.etablissement.find_by_post_etablissement import (
            EtablissementPostMultiCriteres,
        )

        # Search for establishments using the SIREN
        search_criteria = EtablissementPostMultiCriteres(
            q=f"siren:{siren}",
            nombre=100,  # Get up to 100 results
            debut=0,
            masquer_valeurs_nulles=False,  # Show all fields
        )

        print(f"📋 Searching establishments with criteria: q=siren:{siren}")

        # Make the search request
        response = find_by_post_etablissement_sync(body=search_criteria, client=client)

        if hasattr(response, "etablissements") and response.etablissements:
            establishments = response.etablissements
            print(f"✅ Found {len(establishments)} establishments")

            siret_codes = []
            print(f"\n📋 ALL SIRET CODES FOR SIREN {siren} (WITH CORRECT NAMES):")
            print("-" * 60)

            for i, etab in enumerate(establishments, 1):
                siret = etab.siret
                nic = siret[-5:]  # Last 5 digits are the NIC

                # Get the CORRECT facility name from uniteLegale
                facility_name = "Unknown Facility"
                if hasattr(etab, "unite_legale") and etab.unite_legale:
                    if hasattr(etab.unite_legale, "denomination_unite_legale"):
                        facility_name = (
                            etab.unite_legale.denomination_unite_legale
                            or "Unknown Facility"
                        )

                # Get address info
                address_info = "N/A"
                if (
                    hasattr(etab, "adresse_etablissement")
                    and etab.adresse_etablissement
                ):
                    addr = etab.adresse_etablissement
                    street = getattr(addr, "libelle_voie_etablissement", "")
                    city = getattr(addr, "libelle_commune_etablissement", "")
                    postal = getattr(addr, "code_postal_etablissement", "")
                    address_info = f"{street}, {postal} {city}".strip(", ")

                # Get activity info
                activity_info = "N/A"
                if hasattr(etab, "unite_legale") and etab.unite_legale:
                    if hasattr(etab.unite_legale, "activite_principale_unite_legale"):
                        activity_info = (
                            etab.unite_legale.activite_principale_unite_legale or "N/A"
                        )

                print(f"   {i:2d}. SIRET: {siret} (NIC: {nic})")
                print(f"       • Company: {facility_name}")
                print(f"       • Address: {address_info}")
                print(f"       • Activity: {activity_info}")
                print(
                    f"       • Creation: {getattr(etab, 'date_creation_etablissement', 'N/A')}"
                )
                print(
                    f"       • Headquarters: {getattr(etab, 'etablissement_siege', False)}"
                )
                print()

                siret_codes.append(
                    {
                        "siret": siret,
                        "nic": nic,
                        "facility_name": facility_name,
                        "address": address_info,
                        "activity": activity_info,
                        "etablissement": etab,
                    }
                )

            print(f"📈 SUMMARY:")
            print(f"   • SIREN: {siren}")
            print(f"   • Company: {facility_name}")  # All should be the same
            print(f"   • Total SIRET codes found: {len(siret_codes)}")
            print(f"   • NIC codes: {[s['nic'] for s in siret_codes]}")

            return {
                "siret_codes": siret_codes,
                "siren": siren,
                "company_name": facility_name,
            }

        else:
            print(f"❌ No establishments found for SIREN {siren}")
            print(f"Response: {response}")
            return None

    except Exception as e:
        print(f"❌ Error in corrected extraction: {e}")
        return None


# Test with SIREN 061500542
print("🚀 CORRECTED SIRET DISCOVERY")
print("=" * 60)

siren_data_corrected = get_all_siret_from_siren_corrected("061500542")

if siren_data_corrected:
    print(f"\n🎉 SUCCESS! Found {len(siren_data_corrected['siret_codes'])} SIRET codes")
    print(f"📋 Complete list with CORRECT facility names:")
    for siret_info in siren_data_corrected["siret_codes"]:
        print(
            f"   • {siret_info['siret']} (NIC: {siret_info['nic']}) - {siret_info['facility_name']}"
        )
else:
    print(f"\n❌ Corrected extraction failed for SIREN 061500542")

🚀 CORRECTED SIRET DISCOVERY
🔍 CORRECTED SIRET DISCOVERY FOR SIREN: 061500542
📋 Searching establishments with criteria: q=siren:061500542
✅ Found 33 establishments

📋 ALL SIRET CODES FOR SIREN 061500542 (WITH CORRECT NAMES):
------------------------------------------------------------
    1. SIRET: 06150054200040 (NIC: 00040)
       • Company: BDO RHONE-ALPES
       • Address: D'APREMONT, 73190 SAINT-BALDOPH
       • Activity: 69.20Z
       • Creation: 1900-01-01
       • Headquarters: False

    2. SIRET: 06150054200057 (NIC: 00057)
       • Company: BDO RHONE-ALPES
       • Address: FERNAND PELLOUTIER, 38130 ECHIROLLES
       • Activity: 69.20Z
       • Creation: 1900-01-01
       • Headquarters: False

    3. SIRET: 06150054200263 (NIC: 00263)
       • Company: BDO RHONE-ALPES
       • Address: DE LA REPUBLIQUE, 69002 LYON
       • Activity: 69.20Z
       • Creation: 2009-10-01
       • Headquarters: False

    4. SIRET: 06150054200149 (NIC: 00149)
       • Company: BDO RHONE-ALPES
 